In [7]:
#pip install sqlalchemy

In [8]:
#pip install sentence-transformers

In [11]:
#pip install tf-keras

In [2]:
import os
import openai
import json
import tiktoken
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sqlalchemy import create_engine, Column, Integer, String, Text
from sqlalchemy.orm import declarative_base, sessionmaker

In [15]:
os.environ["OPENAI_API_KEY"] = "개인키입력"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [4]:
def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

loader = PyPDFLoader("D:/[24]ICT_Practice/practice_file/(2024)포트미스_가이드북.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, length_function=tiktoken_len)
texts = text_splitter.split_documents(pages)

model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

docsearch = Chroma.from_documents(texts, hf)

openai = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    streaming=False, 
    temperature=0
)

qa = RetrievalQA.from_chain_type(
    llm=openai,
    chain_type="stuff",
    retriever=docsearch.as_retriever(
        search_type="mmr",
        search_kwargs={'k': 3, 'fetch_k': 10}
    ),
    return_source_documents=True
)

C:\ProgramData\Anaconda3\envs\langchainenv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
C:\ProgramData\Anaconda3\envs\langchainenv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
# SQLAlchemy 설정
DATABASE_URL = "sqlite:///conversations.db"
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# 대화 내용을 저장할 테이블 모델 정의
class Conversation(Base):
    __tablename__ = "conversations"

    id = Column(Integer, primary_key=True, index=True)
    query = Column(String, index=True)
    response = Column(Text)

# 테이블 생성
Base.metadata.create_all(bind=engine)

def save_conversation(db_session, query, response):
    conversation = Conversation(query=query, response=response)
    db_session.add(conversation)
    db_session.commit()

def get_response(input_text):
    return qa.invoke({"query": input_text})

In [6]:
# 전체 
def print_all_conversations():
    db_session = SessionLocal()
    conversations = db_session.query(Conversation).all()
    for conv in conversations:
        print(f"ID: {conv.id}, Query: {conv.query}, Response: {conv.response}")
    db_session.close()

# 아이디별
def get_conversation_by_id(conversation_id):
    db_session = SessionLocal()
    conversation = db_session.query(Conversation).filter(Conversation.id == conversation_id).first()
    db_session.close()
    if conversation:
        return {
            "id": conversation.id,
            "query": conversation.query,
            "response": conversation.response
        }
    else:
        return None

In [7]:
#주피터노트북용
def save_query_response(input_text):
    try:
        chat_response = get_response(input_text)
        response_text = chat_response["result"]

        # 대화 내용 저장
        db_session = SessionLocal()
        save_conversation(db_session, input_text, response_text)
        db_session.close()

        output = {
            "status": 200,
            "message": "채팅 응답 성공",
            "body": {
                "chatMessage": response_text
            }
        }
    except Exception as e:
        output = {
            "status": 404,
            "message": "채팅 응답 실패",
            "body": {
                "error": str(e)
            }
        }

    return json.dumps(output, ensure_ascii=False, indent=4)

In [8]:
def show_all_conversations():
    print_all_conversations()

def show_conversation_by_id(conversation_id):
    conversation = get_conversation_by_id(conversation_id)
    if conversation:
        return f"ID: {conversation['id']}, Query: {conversation['query']}, Response: {conversation['response']}"
    else:
        return "Conversation not found."

In [11]:
#예시
input_text = "선박입,출항신고 근거 법령이 뭐야?"
print(save_query_response(input_text))

{
    "status": 200,
    "message": "채팅 응답 성공",
    "body": {
        "chatMessage": "선박의 입출항신고 근거 법령은 선박입출항법 제4조 제1항, 시행령 제2조, 제3조, 시행규칙 제3조, 제4조입니다."
    }
}


In [13]:
# 전체 기록 조회
show_all_conversations()

ID: 1, Query: -f, Response: I'm sorry, but I don't understand your question. Could you please provide more context or clarify what you mean by "-f"?
ID: 2, Query: -f, Response: I'm sorry, but I don't understand your question. Could you please provide more context or clarify what you need help with?
ID: 3, Query: 포트미스 가이드북에 대해 알려줘, Response: 죄송합니다. 포트미스 가이드북에 대한 정보는 제공할 수 없습니다.
ID: 4, Query: 포트미스 가이드북 목차 알려줘, Response: 죄송합니다. 포트미스 가이드북 목차에 대한 정보는 제공드릴 수 없습니다.
ID: 5, Query: 포트미스 가이드북이 뭐야, Response: 포트미스 가이드북은 포트미스 웹사이트를 통해 제공되는 정보를 정리하고 안내하는 책자나 온라인 자료를 말합니다. 이를 통해 포트미스 서비스를 이용하는 사용자들이 원활하게 정보를 찾고 이용할 수 있도록 도와줍니다.
ID: 6, Query: 외항선 입항 신고 어떻게 해?, Response: 외항선 입항 신고는 선박이 무역항의 수상구역 등으로 입항할 때 필요한 절차입니다. 최초에는 선박 입항 전까지 입항 예정 정보를 신고하고, 실제 선박이 입항한 후에 최종적으로 입항을 신고해야 합니다. 이때 승무원/승객명부에 대한 정보도 함께 제출해야 합니다. 이러한 신고는 해양수산부의 PORT-MIS 신고 시스템을 통해 할 수 있습니다.
ID: 7, Query: 외항선 입항 신고 어떻게 해?, Response: 외항선 입항 신고는 선박이 무역항의 수상구역 등으로 입항할 때 필요한 절차입니다. 최초에는 선박 입항 전까지 입항 예정 정보를 신고하고, 실제 선박이 입항한 후에 최종적으로 입항을 신고해야 합

In [14]:
# 각 ID 대화 기록 조회
conversation_id = 7
print(show_conversation_by_id(conversation_id))

ID: 7, Query: 외항선 입항 신고 어떻게 해?, Response: 외항선 입항 신고는 선박이 무역항의 수상구역 등으로 입항할 때 필요한 절차입니다. 최초에는 선박 입항 전까지 입항 예정 정보를 신고하고, 실제 선박이 입항한 후에 최종적으로 입항을 신고해야 합니다. 이때 승무원/승객명부에 대한 정보도 함께 제출해야 합니다. 이러한 신고는 해양수산부의 PORT-MIS 신고 시스템을 통해 할 수 있습니다.
